# Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [2]:
import pandas as pd
from datetime import datetime
movies = pd.read_csv('/Users/maksim/Netology/material/ml-latest-small/movies.csv')
ratings = pd.read_csv('/Users/maksim/Netology/material/ml-latest-small/ratings.csv')

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [38]:
df_over_100 = ratings.groupby('userId').count().query('movieId >= 100')[['movieId']]
df_over_100

,movieId
userId,
1,232
4,216
6,314
7,152
10,140
...,...
605,221
606,1115
607,187


In [39]:
df_100 = ratings.groupby('userId').max()[['timestamp']]
df_100['timestamp_min'] = ratings.groupby('userId').min()[['timestamp']]
#ratings['userId'].value_counts().query('userId >= 100', inplace = True)
#ratings.query('userId >= 100', inplace = True)
df_100


,timestamp,timestamp_min
userId,,
1,965719662,964980499
2,1445715340,1445714835
3,1306464293,1306463323
4,1007574542,945078428
5,847435337,847434747
...,...,...
606,1368460577,1171228846
607,997847387,963078392
608,1189563917,1117161730


In [42]:
#Присоединяем к отфильтрованому датафрейму, фрейм с min и max timestamp
df_final = df_over_100.join(df_100, on = 'userId', how = 'left')
df_final

,movieId,timestamp,timestamp_min
userId,,,
1,232,965719662,964980499
4,216,1007574542,945078428
6,314,845556915,845553109
7,152,1176181731,1106635416
10,140,1455619275,1455301553
...,...,...,...
605,221,1277177012,1277094764
606,1115,1368460577,1171228846
607,187,997847387,963078392


In [32]:
#Функция принимает строку и возвращает Timedelta
def livetime (row):
    max_d = datetime.fromtimestamp(row['timestamp'])
    min_d = datetime.fromtimestamp(row['timestamp_min'])
    return max_d - min_d


In [43]:
#Применим функцию и запишем значения Timedelta в строку, затем посчитаем среднее для столбца
df_final['timestamp_delta'] = df_final.apply(livetime, axis = 1)
df_final.timestamp_delta.mean()

Timedelta('511 days 11:03:29.451612')

# Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [4]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [5]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [6]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [7]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [17]:
client_base[['client_id']].merge(auto, how = 'outer').merge(air, how = 'outer').merge(rzd, how = 'outer')

,client_id,auto_revenue,air_revenue,rzd_revenue
0,111,NaN,NaN,1093.0
1,112,NaN,NaN,2810.0
2,113,57483.0,NaN,10283.0
3,114,83.0,NaN,5774.0
4,115,912.0,81.0,981.0
5,116,4834.0,4.0,NaN
6,117,98.0,13.0,NaN
7,118,NaN,173.0,NaN


In [18]:
client_base.merge(auto, how = 'outer').merge(air, how = 'outer').merge(rzd, how = 'outer')

,client_id,address,auto_revenue,air_revenue,rzd_revenue
0,111,Комсомольская 4,NaN,NaN,1093.0
1,112,Энтузиастов 8а,NaN,NaN,2810.0
2,113,Левобережная 1а,57483.0,NaN,10283.0
3,114,Мира 14,83.0,NaN,5774.0
4,115,ЗЖБИиДК 1,912.0,81.0,981.0
5,116,Строителей 18,4834.0,4.0,NaN
6,117,Панфиловская 33,98.0,13.0,NaN
7,118,Мастеркова 4,NaN,173.0,NaN


# Задание 3
В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?

Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?

Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

# Решение
1) Для решения задачи нам потребуется время, когда эти координаты были получены. Зная время совершения покупки или визита можно синхронизироват таблицу по времени. Каждой покупке будет соответствовать два числа(широта/долгота).
Если время получить затруднительно, то можно передавать последовательно для каждого визита две координаты, первую записывать в широту вторую в долготу. Думаю это можно реализовать через цикл. 

2) Не отразиться на вопросе, поскольку для каждого визита есть своя пара координат. 

3) Название города, населенного пункта, страны, др.